In [ ]:
import json
import math
import os
import subprocess

import ipywidgets as widgets
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.colors import rgb2hex

import horizon_api

MAPA_TELES = "telesa/"

# Simulacija nebesnih teles

## Prenos nebesnih teles
Poženite spodnji blok da lahko ustvatite datoteke za izbrana telsa na datum ki ga vnesete.

In [ ]:
slovar_teles = horizon_api.search_body()
datum = input("zapišite izbran datum v formatu yyyy-mm-dd: ")

text_ime_telesa = widgets.Text(description="Ime telesa",continuous_update=False)
dropdown_izbira = widgets.Dropdown(
    options=[],
    description='Izbira telesa:',
)
out = widgets.Output(layout={'border': '1px solid black', 'padding': "10px"})

# populira dropdown z najdenimi ključi za iskan niz
def najdi_telo(t):
    ime_telesa = str(text_ime_telesa.value).capitalize()
    najdeno = horizon_api.search_body(ime_telesa.strip())
    
    # obravnavajmo primer, ko je najden le en rezultat pri iskanju niza
    if najdeno and "_single_" in najdeno:
        for ime, id_telesa in slovar_teles.items():
            if id_telesa.strip() == najdeno["_single_"].strip():
                najdeno = {ime: id_telesa}
                break
    
    out.clear_output(); dropdown_izbira.value=None; dropdown_izbira.options = []
    if not najdeno: 
        with out: print("Žal nismo našli nobeno telo z tem ključem...") 
    else:
        with out: print("Izberite želeno telo s seznama") 
        dropdown_izbira.options = najdeno.keys()

# na disku ustvari datoteko ki predstavlja izbrano telo
def ustvari_telo(t):
    if t['new'] != t['old'] and t['new']:
        name = t['new']
        json_data = horizon_api.get_body_json(slovar_teles[name], datum, name=name)

        with open(f"{MAPA_TELES}telo_{name}.json", "w", encoding="utf-8") as f:
            json.dump(json_data, f, indent=4)
        out.clear_output()

        if json_data['mass'] == 0 or json_data["radius"] == 0 or len(json_data["x_vec"]) == 0 or len(json_data["v_vec"]) == 0:
            with out: print("Telo ustvarjeno! (manjkajo ključni podatki)")
        else:
            with out: print("Telo ustvarjeno!") 
        

text_ime_telesa.observe(najdi_telo, names="value")
dropdown_izbira.observe(ustvari_telo, names="value")

display(text_ime_telesa); display(dropdown_izbira); display(out)

## Primerjava nebesnih teles po polmeru in velikosti
Prvi blok prenesena telesa parsira in jih srani v seznam, nasledji blok pa za izbrana nariše grafike, ki jih lahko uporabnik prenese.

In [ ]:
# naložimo vsa prenesena telesa in priredimo barve telesom glede na naraščajoč polmer
prenesena_telesa = []

cm = plt.get_cmap('gist_rainbow')
for datoteka in list(os.listdir(MAPA_TELES)):
    with open(os.path.join(MAPA_TELES, datoteka)) as f:
        prenesena_telesa.append(json.load(f))

prenesena_telesa = sorted(prenesena_telesa, key=lambda x: x["radius"])
for i in range(len(prenesena_telesa)):
    prenesena_telesa[i]["color"] = cm(i/len(prenesena_telesa))

In [ ]:
dropdown_grafike = widgets.SelectMultiple(
    options=[(telo["name"], i) for i, telo in enumerate(prenesena_telesa)],
    description='Izbira teles za prikaz grafik',
    rows = 10,
)

btn_narisi = widgets.Button(description='Ustvari grafiki')

# nariše grafiki za polmere in mase
def ustvari_primerjavo(t):
    # izbiro uredimo po velikosti polmerov
    izbira = [prenesena_telesa[i] for i in dropdown_grafike.value]
    izbira = sorted(izbira, key=lambda x: x["radius"])

    # narišemo grafiko velikosti
    graf_velikosti, graf_velikosti_axis = plt.subplots()
    x = 0
    for telo in izbira:
        x += telo["radius"]
        circ = plt.Circle((x,0),telo["radius"], color=telo["color"], label=telo["name"])
        x += telo["radius"]

        graf_velikosti_axis.add_patch(circ)
        graf_velikosti_axis.legend()

    graf_velikosti.set_figwidth(15)
    graf_velikosti_axis.autoscale(); graf_velikosti_axis.set_xticks([])
    graf_velikosti_axis.set_aspect('equal', adjustable='box')

    # narišemo grafiko mase
    graf_mase, graf_mase_axis = plt.subplots()
    bars = graf_mase_axis.bar([telo["name"] for telo in izbira], [math.log(telo["mass"]) for telo in izbira])
    for i, bar in enumerate(bars): bar.set_color(izbira[i]["color"])

    graf_mase.set_figwidth(15); graf_mase_axis.autoscale()
    graf_mase_axis.set_ylabel("20*log(kg)")

btn_narisi.on_click(ustvari_primerjavo)
display(dropdown_grafike); display(btn_narisi)

## 3d prikaz gibanja nebesnih teles
Prvi blok definira metode za simulacijo, zadnji pa za izbrana prenesena telesa nariše animacijo njihovega gibanja v 3d prostoru. 

In [ ]:
# pridobi input za izbrana telesa, centrirano gleda na zadnje telo v seznamu
def pridobi_input(telesa, G, casovni_korak, obdobje_casa, st_korakov):
    stanja_teles = [f"{telo['mass']} {telo['x_vec'][0]} {telo['x_vec'][1]} {telo['x_vec'][2]} {telo['v_vec'][0]} {telo['v_vec'][1]} {telo['v_vec'][2]}" for telo in telesa]
    return f"{G} {casovni_korak} {obdobje_casa} {st_korakov} {len(stanja_teles)} {' '.join(stanja_teles)}"

def simuliraj(telesa, G, casovni_korak, obdobje_casa, st_korakov):
    inp = pridobi_input(telesa, G, casovni_korak, obdobje_casa, st_korakov)
    
    # poženemo simulacijo
    proces_simulacije = subprocess.Popen(
        ["./simulacija"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
    )
    stdout = proces_simulacije.communicate(input=inp.encode("utf-8"))[0].decode("utf-8").split("\n")

    # parsiramo output v lepšo obliko za vsako telo posebaj
    podatki_simulacije = [[] for _ in range(len(telesa))]
    for i in range(st_korakov):
        for j in range(len(telesa)):
            podatki_simulacije[j].append(tuple(
                map(float,
                    stdout[i*len(telesa)+j].strip().split()
                )
            ))

    return podatki_simulacije

In [ ]:
# število korakov časa v prikazu simulacije
koraki = 100

dropdown_simulacija = widgets.SelectMultiple(
    options=[(telo["name"], i) for i, telo in enumerate(prenesena_telesa)],
    description='Izbira teles za simulacijo',
    rows = 10,
    style={'description_width': 'initial'}
)

text_G = widgets.FloatText(value=6.674, description='Gravitacijska konstanta * 10^(-11): ', style={'description_width': 'initial'})
text_casovni_korak = widgets.FloatText(value=60, description='Časovni korak (v sekundah): ', style={'description_width': 'initial'})
text_obdobje_casa_d = widgets.FloatText(value=365, description='Obdobje simulacije (v dnevih): ', style={'description_width': 'initial'})
text_obdobje_casa_s = widgets.FloatText(value=0, description='Obdobje simulacije (v sekundah): ', style={'description_width': 'initial'})

slider_dolzina_slicice = widgets.IntSlider(value=40, min=1, max=200, step=1, description='Dolžina sličice (v ms): ', style={'description_width': 'initial'})
btn_simuliraj = widgets.Button(description='Simuliraj telesa')

def ustvari_prikaz_simulacije(t):
    # izbiro uredimo po masi, da bo telo z največjo maso center simulacije 
    izbira = [prenesena_telesa[i] for i in dropdown_simulacija.value]
    izbira = sorted(izbira, key=lambda x: x["mass"])

    barve_izbire = list(map(lambda t: rgb2hex(t["color"]), izbira))

    # pridobimo simulirano gibanje izbranih teles
    G,dt = float(text_G.value), float(text_casovni_korak.value)
    t,dframe = float(text_obdobje_casa_d.value)*24*3600+float(text_obdobje_casa_s.value), float(slider_dolzina_slicice.value)
    
    podatki = simuliraj(izbira, G, dt, t, koraki)

    # ustvarimo prazno figuro za prikaz simulacije
    prikaz_simulacije = go.Figure(
        layout=go.Layout(
            scene=dict(
                aspectmode='cube',
                aspectratio=dict(x=1, y=1, z=1)
            )
        ), 
        data=[go.Scatter3d() for _ in range(len(izbira)) ]
    )

    # dodamo časovne korake v prikaz simulacije in sledimo maksimalni oddaljenosti teles
    frames = []; max_razdalja = 0
    for t in range(koraki):
        data = []
        for i,podatki_telesa in enumerate(podatki):
            data.append(go.Scatter3d(
                x=[podatki_telesa[t][0]],
                y=[podatki_telesa[t][1]],
                z=[podatki_telesa[t][2]],
                name=izbira[i]["name"],
                mode="markers",
                marker=dict(color=barve_izbire[i])
            ))
            max_razdalja = max(max(abs(max(podatki_telesa[t])), abs(min(podatki_telesa[t]))), max_razdalja)

        frames.append(go.Frame(data=data))

    # dodamo časovne korake v simulacijo in dodamo gumbe v layout
    prikaz_simulacije.update(frames=frames)
    prikaz_simulacije.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="Predvajaj simulacijo", method="animate", args=[None, dict(frame=dict(redraw=True, duration=dframe), mode='immediate', fromcurrent=True)]),
                    dict(label="Ustavi simulacijo", method="animate", args=[[None], dict(frame=dict(redraw=True, duration=0), mode='immediate', fromcurrent=True)])
                ]
            )
        ]
    )

    # omejimo velikost prikaza
    prikaz_simulacije['layout']['scene']['xaxis']['range'] = [-1.2*max_razdalja, 1.2*max_razdalja]
    prikaz_simulacije['layout']['scene']['yaxis']['range'] = [-1.2*max_razdalja, 1.2*max_razdalja]
    prikaz_simulacije['layout']['scene']['zaxis']['range'] = [-1.2*max_razdalja, 1.2*max_razdalja]
    
    prikaz_simulacije.show()
    
btn_simuliraj.on_click(ustvari_prikaz_simulacije)

display(dropdown_simulacija)
display(widgets.Box(children=[text_G, text_casovni_korak]))
display(widgets.Box(children=[text_obdobje_casa_d, text_obdobje_casa_s])) 
display(slider_dolzina_slicice); display(btn_simuliraj)